In [1]:
import torch
import torch_harmonics as th
import numpy as np
import os 
import h5py
import glob
import sys

from varsStormer import varsStormer
from stormer_utils_pangu import StormerWrapperPangu

/home/mjp5595/.conda/envs/climaX/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")
/home/mjp5595/.conda/envs/climaX/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [15]:
def read_era5(data,vars_stormer):
    data_np = np.zeros((len(vars_stormer),128,256))
    for i,var in enumerate(vars_stormer):
        data_np[i] = data['input/{}'.format(var)][:]
    return data_np

def norm_era5(data_np,vars_stormer,stormer_wrapper):
    data_torch = torch.from_numpy(data_np)
    data_torch_norm = stormer_wrapper.inp_transform(data_torch)
    data_np_norm = data_torch_norm.numpy()
    return data_np_norm

In [16]:
forecast_dir = '/eagle/MDClimSim/mjp5595/data/stormer/stormer_forecasts_2017_norm/'
era5_dir = '/eagle/MDClimSim/tungnd/data/wb2/1.40625deg_from_full_res_1_step_6hr_h5df/train/'

vars_stormer = varsStormer().vars_stormer

stormer_wrapper = StormerWrapperPangu(
    root_dir='/eagle/MDClimSim/tungnd/data/wb2/1.40625deg_from_full_res_1_step_6hr_h5df/',
    variables=vars_stormer,
    net=None,
    base_lead_time=[6],
    ckpt=None,
    device=None,
)

checkpoint model loaded


In [22]:
forecasts_raw = h5py.File(os.path.join(forecast_dir,'raw_0000.h5'),'r')
forecasts_norm = h5py.File(os.path.join(forecast_dir,'norm_0000.h5'),'r')
gt_0 = h5py.File(os.path.join(era5_dir,'2017_0000.h5'),'r')
gt_6 = h5py.File(os.path.join(era5_dir,'2017_0001.h5'),'r')

In [23]:
# This Validates the forecasts in /stormer/stormer_forecasts_2017_norm/ vs ground truth

diff_init = forecasts_raw['0'][:] - read_era5(gt_0,vars_stormer)
print('diff_init :',np.max(diff_init))
diff_pred_vs_gt = forecasts_raw['6'][:] - read_era5(gt_6,vars_stormer)
print('diff_pred_vs_gt :',np.max(diff_pred_vs_gt))
diff_pred_vs_init = forecasts_raw['6'][:] - forecasts_raw['0'][:]
print('diff_pred_vs_init :',np.max(diff_pred_vs_init))

diff_init : 0.0
diff_pred_vs_gt : 305.671875
diff_pred_vs_init : 2019.2266
